In [1]:
%matplotlib inline
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from metpy.units import units
from netCDF4 import num2date, Dataset
import numpy as np
from siphon.ncss import NCSS
import metpy.calc as mpcalc
from scipy.ndimage import gaussian_filter

import warnings
warnings.filterwarnings('ignore')

In [2]:
def getdata(year='1999',month='6', day='19', hour='12'):
    
    # Reduce lat/lon to get rid of missing values
    iymin = 5
    #iymin = 1
    ixmin = 5
    #ixmin = 1
    iymax = 270
    ixmax = 335
    
    dt = datetime(int(year),int(month),int(day),int(hour))
    
    # Grab Pressure Level Data
    hght_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/hgt.{0:%Y%m}.nc'.format(dt))
    air_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                               'NARR/pressure/air.{0:%Y%m}.nc'.format(dt))
    shum_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/shum.{0:%Y%m}.nc'.format(dt))
    uwnd_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/uwnd.{0:%Y%m}.nc'.format(dt))
    vwnd_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/vwnd.{0:%Y%m}.nc'.format(dt))
        
    vtimes = num2date(hght_data.variables['time'][:], units='hours since 1800-1-1 00:00:0.0')
    itime = np.where(vtimes==dt)[0][0]
    
    data = {'Geopotential_Height':hght_data.variables['hgt'][itime,:,iymin:iymax,ixmin:ixmax],
            'Air_Temperature':air_data.variables['air'][itime,:,iymin:iymax,ixmin:ixmax],
            'Specific_Humidity':shum_data.variables['shum'][itime,:,iymin:iymax,ixmin:ixmax],
            'U_wind':uwnd_data.variables['uwnd'][itime,:,iymin:iymax,ixmin:ixmax],
            'V_wind':vwnd_data.variables['vwnd'][itime,:,iymin:iymax,ixmin:ixmax],
            'lat':hght_data.variables['lat'][iymin:iymax,ixmin:ixmax],
            'lon':hght_data.variables['lon'][iymin:iymax,ixmin:ixmax],
            'time':vtimes[itime],
            'level':hght_data.variables['level']}
    return data

In [11]:
def plot(level='300'):
    data = x.widget.result
    year = x.widget.kwargs['year']
    month = x.widget.kwargs['month']
    day = x.widget.kwargs['day']
    hour = x.widget.kwargs['hour']
    
    
    dt = datetime(int(year),int(month),int(day),int(hour))
    
    # Pull out the lat and lon data
    lat = data['lat'][:]
    lon = data['lon'][:]

    plotcrs = ccrs.LambertConformal(central_latitude=45., central_longitude=-100.,
                                    standard_parallels=[30, 60])
    
    datacrs = ccrs.PlateCarree()

    tlatslons = plotcrs.transform_points(datacrs,lon,lat)
    tlon = tlatslons[:,:,0]
    tlat = tlatslons[:,:,1]
    


    
    preslevs = units('hPa') * data['level'][:]
    temp_var = gaussian_filter(data['Air_Temperature'][:], sigma=1.0) * units.kelvin
    shum_var = gaussian_filter(data['Specific_Humidity'][:], sigma=1.0) * units('kg/kg')
    uwnd_var = gaussian_filter(data['U_wind'][:], sigma=1.0) * units('m/s')
    vwnd_var = gaussian_filter(data['V_wind'][:], sigma=1.0) * units('m/s')
    #hght_var = data['Geopotential_Height'][:] * units.meter
    
    isentlevs = [float(level)] * units.kelvin
    isent_anal = mpcalc.isentropic_interpolation(isentlevs,
                                                 preslevs,
                                                 temp_var,
                                                 shum_var,
                                                 uwnd_var,
                                                 vwnd_var,
                                                 tmpk_out=True)
    
    isent_pres = isent_anal[0]
    isent_temp = isent_anal[1]
    isent_shum = isent_anal[2]
    isent_uwnd = isent_anal[3]
    isent_vwnd = isent_anal[4]
    #isent_hght = isent_anal[5]
    
    e = mpcalc.vapor_pressure(isent_pres[0],isent_shum[0])
    es = mpcalc.saturation_vapor_pressure(isent_temp[0])
    rel_hum = e/es*100
    rel_hum[rel_hum>100] = 100
    
    fig = plt.figure(figsize=(16, 14))
    ax = fig.add_subplot(111, projection=plotcrs)
    ax.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    #ax.set_title('NARR Plot', size=16)

    # Add state boundaries to plot
    states_provinces = cfeature.NaturalEarthFeature(category='cultural',
                                                    name='admin_1_states_provinces_lakes',
                                                    scale='50m', facecolor='none')
    ax.add_feature(states_provinces, edgecolor='black', linewidth=1)

    # Add country borders to plot
    country_borders = cfeature.NaturalEarthFeature(category='cultural',
                                                   name='admin_0_countries',
                                                   scale='50m', facecolor='none')
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)
        
    # Contour based on variable chosen
    clevisent = np.arange(0, 1001, 50)
    cs = ax.contour(tlon, tlat, isent_pres[0],clevisent,
                    linewidth=2, colors='black')
    plt.clabel(cs,inline=True,fmt='%d')
        
    cf = ax.contourf(tlon, tlat, rel_hum, range(50,101,5), cmap='Greens')
    plt.title(r''+str(level)+'K Isentropic Pressure (hPa), Relative Humidity (%), and Wind Barbs (kts)',
              loc='left')
    
    cb = fig.colorbar(cf, orientation='horizontal',ax=ax,pad=0,aspect=50,extendrect='True')
        
    # Plot Wind Barbs
    # Transform Vectors and plot wind barbs.
    ax.barbs(lon, lat, isent_uwnd[0].to('kts').m, isent_vwnd[0].to('kts').m, length=6, regrid_shape=20, pivot='middle',
             transform=datacrs)

    
    plt.title('VALID: {}'.format(data['time']), loc='right')
    
    plt.show()



In [4]:
year_widget = widgets.Dropdown(
    options=list(range(1979,2017)),
    description='Year', alignment='center')

month_widget = widgets.Dropdown(
    options=list(range(1,13)),
    description='Month', alignment='center')

day_widget = widgets.Dropdown(description='Day', options=list(range(1,32)))
hour_widget = widgets.Dropdown(description='Hour', options=[0,3,6,9,12,15,18,21])


level_widget = widgets.BoundedIntText(value=300,min=255,max=385,step=1,
                                      description='Level: ',disabled=False)


In [5]:
x = widgets.interact_manual(getdata,year=year_widget,month=month_widget, day=day_widget, hour=hour_widget)

interactive(children=(Dropdown(description='Year', options=(1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 19…

In [12]:
print('Input the desired Isentropic Level (valid level: 255-385K)')
p = widgets.interact_manual(plot, level=level_widget)

Input the desired Isentropic Level (valid level: 255-385K)


interactive(children=(BoundedIntText(value=300, description='Level: ', max=385, min=255), Button(description='…